In [278]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import itertools
import re
from tqdm.notebook import tqdm
import time
import random
import pickle
import pandas as pd
from pathlib import Path

In [58]:
# на этот раз в качестве источника данных я выбрал онлайн галлерею gallerix.ru, 
# так как они обладают очень большой коллекцией картин, каталогизированных по художникам

In [59]:
# страница со ссылками на художников

In [60]:
base_url = "https://gallerix.ru/"

In [124]:
def get_all_links():
    res = requests.get(base_url + "storeroom/",  headers={'User-Agent': UserAgent().chrome})
    main_page = res.content
    main_page = BeautifulSoup(main_page, 'html.parser')
    items = main_page.findAll("p", attrs={"class":"sr-pntrs"})
    links = [i["href"] for i in itertools.chain.from_iterable([i.findAll("a") for i in items])]
    return links
links = get_all_links()

In [171]:
def get_data_about_artist(link):
    res = requests.get(base_url + link[1:],  headers={'User-Agent': UserAgent().chrome})
    main_page = res.content
    main_page = BeautifulSoup(main_page, 'html.parser')
    items = main_page.findAll("p", attrs={"class":"sr-pntrs"})
    title_data = main_page.find("h1",  attrs={"class":"panel-title"})
    artist_name = title_data.find("b").text
    try:
        artist_dates = re.findall("\d{4}", title_data.find("span").find("b").text)
        live_from = artist_dates[0]    
        live_to = artist_dates[1]
    except:
        live_from = 0
        live_to = 0
    # удалим часть документа, содержащую картины из других альбомов (сборников) для избежания дублей    
    for el in main_page.findAll("h3"):
        if "в других альбомах" in el.text:
            el.parent.decompose()    
    pictures_list = [i.find("a")["href"] for i in main_page.findAll("div", attrs={"class":"pic"})]
    return {"artist_name":artist_name,
            "live_from":live_from,
            "live_to":live_to,
            "pictures_count":len(pictures_list),
            "pictures_list":pictures_list
           }

In [148]:
# пора загрузить данные о каждом художнике

In [174]:
data = []
failed = []
for artist_link in tqdm(links):
    time.sleep(random.randint(1,2))
    try:
        data.append(get_data_about_artist(artist_link))
    except:
        failed.append(artist_link)
print("Обработано {} записей. Обнаружено {} ошибок".format(len(links), len(failed)))
    
    


Обработано 238 записей. Обнаружено 9 ошибок


In [157]:
# ручные проверки показали что часть ссылок битые (возвращают ошибку 404)

In [190]:
# сохраним спарсенные метаданные на всякий случай
with open("artists_with_pictures.pkl", "wb") as f:
    pickle.dump(data, f)   

In [189]:
with open("artists_with_pictures.pkl", "rb") as f:
    data = pickle.load(f)

In [223]:
def get_picture_details(link):
    res = requests.get(base_url + link[1:],  headers={'User-Agent': UserAgent().chrome})
    main_page = res.content
    main_page = BeautifulSoup(main_page, 'html.parser')
    picture_name = main_page.find("h1", attrs={"class":"panel-title"}).contents[0].strip()
    img_url = main_page.find("p", attrs={"class":"xpic"}).find("img")["src"]
    return picture_name, img_url

In [242]:
data2 = []
for artist in tqdm(data[:]):
    for picture in tqdm(artist["pictures_list"][:], desc = artist["artist_name"] ):
#         time.sleep(random.randint(1,2))
        time.sleep(0.2)
        try:
            picture_name, img_url =  get_picture_details(picture)
        except:
            picture_name, img_url = "unknown", "unknown"
            time.sleep(1)
        
        data2.append({
            "artist_name": artist["artist_name"],
            "live_from": artist["live_from"],
            "live_to": artist["live_to"],
            "pictures_count": artist["pictures_count"],
            "picture_name": picture_name,
            "img_url": img_url,
        })
        

In [245]:
# сохраним результаты труда
pd.DataFrame(data2).to_pickle("paintings_links.pkl")

In [246]:
paintings_links = pd.read_pickle("paintings_links.pkl")

In [259]:
paintings_links[paintings_links.picture_name != "unknown"]

,artist_name,live_from,live_to,pictures_count,picture_name,img_url
0,Иван Константинович Айвазовский,1817,1900,516,Девятый вал,https://sr.gallerix.ru/_EX/1016087178/71308039...
1,Иван Константинович Айвазовский,1817,1900,516,Лунная дорожка,https://sr.gallerix.ru/_EX/1819898631/33996887...
2,Иван Константинович Айвазовский,1817,1900,516,Штиль 1885 42х62,https://sr.gallerix.ru/_EX/1819898631/52061157...
3,Иван Константинович Айвазовский,1817,1900,516,Закат 1866 46х61,https://sr.gallerix.ru/_EX/1819898631/31920471...
4,Иван Константинович Айвазовский,1817,1900,516,Волна 1889 304х505,https://sr.gallerix.ru/_EX/1819898631/59644470...
...,...,...,...,...,...,...
49254,Сергей Арсеньевич Виноградов,1869,1938,111,Пристань на Волге. 1885,https://sr.gallerix.ru/_RUS/637749369/75230970...
49255,Сергей Арсеньевич Виноградов,1869,1938,111,Пристань. Архангельск. 1903,https://sr.gallerix.ru/_RUS/637749369/33525229...
49256,Сергей Арсеньевич Виноградов,1869,1938,111,Этюд с лодкой,https://sr.gallerix.ru/_RUS/637749369/14122712...
49257,Сергей Арсеньевич Виноградов,1869,1938,111,Баба. 1890,https://sr.gallerix.ru/_RUS/637749369/80091214...


In [260]:
# пришло время скачать сами картины

In [305]:
paintings_links["img_path"] = "unknown"

In [329]:
p_name = t.picture_name.replace("?","")

'Бабы тульские. 1889'

In [334]:
p_name[:100]

'Сивилла (Хуана Пачеко)'

In [339]:
for t in tqdm(paintings_links.itertuples(), total=paintings_links[paintings_links.img_path == "unknown"].shape[0]):
    if t.picture_name != "unknown" and t.img_path == "unknown":
        (Path.cwd()/"data"/t.artist_name).mkdir(parents=True, exist_ok=True)
        # много названий картин содержит недопустимые символы
        p_name = t.picture_name.replace("?","").replace("/","").replace('"',"").replace(':',"")[:100]
        image_path = "data/{}/{}_{}.jpg".format(t.artist_name,t.Index, p_name)
        try:
            r = requests.get(t.img_url,  headers={'User-Agent': UserAgent().chrome})
            out = open(str(image_path),'wb')
            out.write(r.content)
            out.close()
            time.sleep(0.3)
            paintings_links.loc[t.Index, "img_path"] = image_path
        except Exception as e:
            print(e)
            pass
    
    

In [342]:
paintings_links.to_pickle("final_dataframe.pkl")